# Notebook 2: Pembuatan Embedding dan Indexing ke ChromaDB (Multi-PDF)

Tujuan notebook ini adalah untuk:
1. Mengambil chunks teks yang telah diproses dari Notebook 1 (dari semua PDF).
2. Menginisialisasi model embedding (OpenAI).
3. Membuat embedding untuk setiap chunk.
4. Menginisialisasi ChromaDB sebagai vector store.
5. Menyimpan (mengindeks) chunks beserta embeddingnya ke ChromaDB.
6. Melakukan tes pencarian sederhana.

## 1. Import library

In [9]:
import os
import pickle
import shutil
from dotenv import load_dotenv
import tiktoken

from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

In [2]:
load_dotenv()

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
openai_api_key_standard = os.getenv("OPENAI_API_KEY")

if not all([azure_openai_endpoint, azure_openai_api_key, azure_openai_embedding_deployment]):
    print("ERROR: Satu atau lebih variabel environment Azure OpenAI (ENDPOINT, API_KEY, EMBEDDING_DEPLOYMENT_NAME) tidak ditemukan.")
    print("Pastikan file .env Anda sudah dikonfigurasi dengan benar untuk Azure.")
else:
    print("Variabel environment Azure OpenAI berhasil dimuat.")

if not openai_api_key_standard:
    print("INFO: OPENAI_API_KEY (standar) tidak ditemukan. Ini mungkin dibutuhkan untuk LLM di Notebook 3 jika tidak menggunakan Azure LLM.")
else:
    print("OpenAI API Key (standar) berhasil dimuat.")

Variabel environment Azure OpenAI berhasil dimuat.
OpenAI API Key (standar) berhasil dimuat.


In [3]:
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
chunk_dir = os.path.join(current_dir, "chunk_files")
chunk_file_names = [file for file in os.listdir(chunk_dir) if file.lower().endswith(".pkl")]

In [4]:
all_chunks = [] # Inisialisasi
chunks_file_path_to_load = None # Path ke file pickle yang akan dimuat

print(f"Mencari direktori chunks di: {chunk_dir}")

if os.path.isdir(chunk_dir):
    try:
        # Dapatkan daftar semua file .pkl di dalam chunk_dir
        chunk_file_names = [file for file in os.listdir(chunk_dir) if file.lower().endswith(".pkl")]
        print(f"File .pkl yang ditemukan di {chunk_dir}: {chunk_file_names}")

        if chunk_file_names:
            # Memuat file .pkl pertama yang ditemukan
            file_to_load = chunk_file_names[0]
            chunks_file_path_to_load = os.path.join(chunk_dir, file_to_load)
            print(f"Akan mencoba memuat file chunks: {chunks_file_path_to_load}")
        else:
            print(f"Tidak ada file .pkl yang ditemukan di direktori {chunk_dir}.")

    except FileNotFoundError:
        print(f"ERROR: Direktori chunks {chunk_dir} tidak ditemukan saat mencoba listdir.")
    except Exception as e:
        print(f"Error saat mengakses direktori chunks {chunk_dir}: {e}")
else:
    print(f"ERROR: Direktori chunks {chunk_dir} tidak ditemukan.")


if chunks_file_path_to_load and os.path.exists(chunks_file_path_to_load):
    try:
        with open(chunks_file_path_to_load, "rb") as f:
            all_chunks = pickle.load(f)
        print(f"Berhasil memuat {len(all_chunks)} chunks dari {chunks_file_path_to_load}")
        if all_chunks:
            print(f"Contoh metadata chunk pertama yang dimuat: {all_chunks[0].metadata}")
    except Exception as e:
        print(f"Error saat memuat chunks dari pickle {chunks_file_path_to_load}: {e}")
        all_chunks = []
elif chunks_file_path_to_load:
     print(f"File chunks {chunks_file_path_to_load} terdefinisi tapi tidak ditemukan.")
else:
    print(f"Tidak ada file pickle yang valid untuk dimuat berdasarkan kriteria.")


if not all_chunks:
    print("\nPERINGATAN: `all_chunks` kosong atau tidak berhasil dimuat.")
    print("Pastikan Notebook 1 (yang sudah diupdate) telah dijalankan dan menyimpan outputnya dengan benar")
    print(f"di dalam direktori '{chunk_dir}'.")

Mencari direktori chunks di: d:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\notebooks\chunk_files
File .pkl yang ditemukan di d:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\notebooks\chunk_files: ['processed_chunks_multi_pdf.pkl']
Akan mencoba memuat file chunks: d:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\notebooks\chunk_files\processed_chunks_multi_pdf.pkl
Berhasil memuat 206 chunks dari d:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\notebooks\chunk_files\processed_chunks_multi_pdf.pkl
Contoh metadata chunk pertama yang dimuat: {'producer': 'Adobe PDF Library 9.0', 'creator': 'Adobe InDesign CS4 (6.0)', 'creationdate': '2020-06-03T12:10:59+05:30', 'moddate': '2020-06-03T12:11:10+05:30', 'source': 'ojsadmin,+207.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}


In [5]:
if all_chunks:
    try:
        # Inisialisasi tokenizer yang sesuai (cl100k_base untuk model seperti text-embedding-ada-002, text-embedding-3-small)
        # Jika Anda menggunakan model embedding yang sangat berbeda, Anda mungkin perlu tokenizer yang berbeda.
        tokenizer = tiktoken.get_encoding("cl100k_base")
        print("Tokenizer 'cl100k_base' berhasil dimuat.")

        total_tokens = 0
        for i, chunk_doc in enumerate(all_chunks):
            if isinstance(chunk_doc, Document) and hasattr(chunk_doc, 'page_content'):
                text_content = chunk_doc.page_content
                tokens = tokenizer.encode(text_content)
                total_tokens += len(tokens)
                # (Opsional) Print token count per chunk untuk debugging
                # if i < 5: # Tampilkan untuk 5 chunk pertama
                #     print(f"Chunk {i+1}: {len(tokens)} tokens")
            else:
                print(f"Peringatan: Item ke-{i} dalam all_chunks bukan objek Document yang valid atau tidak memiliki page_content.")

        print(f"\nTotal Estimasi Token untuk semua {len(all_chunks)} chunks: {total_tokens} tokens")

        # (Opsional) Estimasi Biaya Sederhana (contoh untuk text-embedding-ada-002)
        # Harga dapat berubah, selalu cek pricing resmi OpenAI/Azure.
        # Contoh harga: $0.0001 per 1K tokens untuk text-embedding-3-small (per Mei 2024, bisa berbeda di Azure)
        # Contoh harga: $0.0004 per 1K tokens untuk text-embedding-ada-002 (OpenAI)
        cost_per_1k_tokens_ada_002 = 0.0004
        cost_per_1k_tokens_text_embedding_3_small = 0.0001 # Periksa harga Azure yang berlaku untuk deployment Anda

        estimated_cost_ada_002 = (total_tokens / 1000) * cost_per_1k_tokens_ada_002
        estimated_cost_text_embedding_3_small = (total_tokens / 1000) * cost_per_1k_tokens_text_embedding_3_small

        print(f"Estimasi biaya embedding (text-embedding-ada-002 @ ${cost_per_1k_tokens_ada_002}/1k tokens): ${estimated_cost_ada_002:.6f}")
        print(f"Estimasi biaya embedding (text-embedding-3-small @ ${cost_per_1k_tokens_text_embedding_3_small}/1k tokens): ${estimated_cost_text_embedding_3_small:.6f}")
        print("PERHATIAN: Estimasi biaya ini adalah perkiraan kasar. Selalu periksa harga resmi dari penyedia layanan Anda (OpenAI/Azure).")

    except Exception as e:
        print(f"Error saat menghitung token: {e}")
        print("Pastikan library 'tiktoken' sudah terinstal (`pip install tiktoken`).")
else:
    print("Tidak ada chunks untuk dihitung tokennya.")

Tokenizer 'cl100k_base' berhasil dimuat.

Total Estimasi Token untuk semua 206 chunks: 53886 tokens
Estimasi biaya embedding (text-embedding-ada-002 @ $0.0004/1k tokens): $0.021554
Estimasi biaya embedding (text-embedding-3-small @ $0.0001/1k tokens): $0.005389
PERHATIAN: Estimasi biaya ini adalah perkiraan kasar. Selalu periksa harga resmi dari penyedia layanan Anda (OpenAI/Azure).


In [6]:
embeddings_model = None # Inisialisasi
# Hanya lanjut jika konfigurasi Azure ada dan chunks berhasil dimuat
if all([azure_openai_endpoint, azure_openai_api_key, azure_openai_embedding_deployment]) and all_chunks:
    try:
        embeddings_model = AzureOpenAIEmbeddings(
            azure_endpoint=azure_openai_endpoint,
            openai_api_key=azure_openai_api_key,
            azure_deployment=azure_openai_embedding_deployment,
            openai_api_version=azure_openai_api_version,
            # chunk_size=16 # Opsional: LangChain akan menangani chunking untuk embedding jika teks terlalu panjang
                           # Namun, model Azure memiliki batasan jumlah input (misal, 16 untuk text-embedding-ada-002 jika dipanggil langsung)
                           # LangChain AzureOpenAIEmbeddings menangani batching secara internal.
                           # Untuk text-embedding-3-small, batas input per request adalah 2048 token, dan bisa handle array hingga 2048 input.
        )
        print(f"Model Azure OpenAI Embeddings ('{azure_openai_embedding_deployment}') berhasil diinisialisasi.")

        # (Opsional) Tes embedding pada teks kecil
        # sample_text = "Ini adalah teks contoh untuk Azure embedding."
        # sample_embedding = embeddings_model.embed_query(sample_text)
        # print(f"\nContoh embedding untuk '{sample_text}':")
        # print(sample_embedding[:10]) # Tampilkan 10 elemen pertama dari vektor embedding
        # print(f"Dimensi embedding: {len(sample_embedding)}")

    except Exception as e:
        print(f"Error saat menginisialisasi AzureOpenAIEmbeddings: {e}")
        embeddings_model = None
else:
    print("Tidak dapat menginisialisasi model embedding Azure karena konfigurasi environment atau data chunks tidak ada.")


Model Azure OpenAI Embeddings ('text-embedding-3-small') berhasil diinisialisasi.


In [10]:
vector_store = None # Inisialisasi
if embeddings_model and all_chunks: # Pastikan embeddings_model (Azure) berhasil diinisialisasi
    # `project_root` sudah didefinisikan di sel sebelumnya
    vector_store_dir = os.path.join(project_root, "vector_store", "chroma_db_azure_multi") # Nama direktori baru untuk Azure
    collection_name = "rag_azure_multi_pdf_collection" # Nama koleksi baru untuk Azure

    print(f"Akan menggunakan direktori untuk ChromaDB (Azure): {vector_store_dir}")
    print(f"Nama koleksi (Azure): {collection_name}")

    # OPSI UNTUK MEMBERSIHKAN DATABASE LAMA ( uncomment jika ingin selalu mulai baru)
    # HATI-HATI: Ini akan menghapus semua data di `vector_store_dir`!
    clean_start = True # Set ke True untuk menghapus DB lama
    if clean_start and os.path.exists(vector_store_dir):
        print(f"PEMBERSIHAN: Menghapus direktori ChromaDB lama: {vector_store_dir}")
        try:
            shutil.rmtree(vector_store_dir)
            print("Direktori ChromaDB lama berhasil dihapus.")
        except Exception as e:
            print(f"Error saat menghapus direktori ChromaDB lama: {e}")
    
    os.makedirs(vector_store_dir, exist_ok=True) # Pastikan direktori ada

    try:
        # Cara paling sederhana adalah menggunakan from_documents jika ini adalah proses indexing utama
        print(f"Membuat (atau menimpa) vector store dengan {len(all_chunks)} chunks menggunakan Azure Embeddings...")
        vector_store = Chroma.from_documents(
            documents=all_chunks,
            embedding=embeddings_model, # Menggunakan Azure embeddings_model
            collection_name=collection_name,
            persist_directory=vector_store_dir
        )
        vector_store.persist() # Penting untuk menyimpan perubahan ke disk
        print("ChromaDB vector store berhasil dibuat/diperbarui dan di-persist dengan data baru (Azure).")

    except Exception as e:
        print(f"Error saat menginisialisasi atau mengisi ChromaDB dengan Azure Embeddings: {e}")
        vector_store = None
else:
    print("Tidak dapat menginisialisasi ChromaDB karena model embedding Azure atau data chunks tidak ada.")


Akan menggunakan direktori untuk ChromaDB (Azure): d:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\vector_store\chroma_db_azure_multi
Nama koleksi (Azure): rag_azure_multi_pdf_collection
PEMBERSIHAN: Menghapus direktori ChromaDB lama: d:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\vector_store\chroma_db_azure_multi
Error saat menghapus direktori ChromaDB lama: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\Zulfi\\CodeLabs\\SeiZen\\SeiZen-RAG\\vector_store\\chroma_db_azure_multi\\308e9d01-6da8-41c8-bb54-c62f1612ecd0\\data_level0.bin'
Membuat (atau menimpa) vector store dengan 206 chunks menggunakan Azure Embeddings...
ChromaDB vector store berhasil dibuat/diperbarui dan di-persist dengan data baru (Azure).


C:\Users\Zulfi\AppData\Local\Temp\ipykernel_15624\2832737120.py:32: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist() # Penting untuk menyimpan perubahan ke disk


In [11]:
if vector_store:
    query_text = "what is difference between superviced and self-superviced learning?" # Ganti dengan query yang relevan dengan isi PDF Anda
    print(f"\nMelakukan similarity search untuk query: '{query_text}' (menggunakan Azure Embeddings)")
    try:
        search_results = vector_store.similarity_search(query_text, k=3)

        if search_results:
            print(f"\nDitemukan {len(search_results)} hasil yang relevan:")
            for i, doc_result in enumerate(search_results):
                print(f"\n--- Hasil Pencarian {i+1} ---")
                source_file = doc_result.metadata.get('source', 'Tidak diketahui')
                page_number = doc_result.metadata.get('page', 'N/A')
                print(f"Sumber: {source_file}, Halaman: {page_number}")
                print(f"Konten: {doc_result.page_content[:300]}...")
        else:
            print("Tidak ada hasil yang ditemukan untuk query tersebut.")
    except Exception as e:
        print(f"Error saat melakukan similarity search: {e}")
else:
    print("Tidak dapat melakukan pencarian karena vector store tidak terinisialisasi.")


Melakukan similarity search untuk query: 'what is difference between superviced and self-superviced learning?' (menggunakan Azure Embeddings)

Ditemukan 3 hasil yang relevan:

--- Hasil Pencarian 1 ---
Sumber: Self-superviced Learning.pdf, Halaman: 7
Konten: As stated in [123], contrastive models tend to be data-
hungry and vulnerable to overfitting issues, whereas gen-
erative models encounter data-filling challenges and ex-
hibit inferior data scaling capabilities when compared to
contrastive models. While contrastive models often fo-
cus on global vi...

--- Hasil Pencarian 2 ---
Sumber: Self-superviced Learning.pdf, Halaman: 7
Konten: As stated in [123], contrastive models tend to be data-
hungry and vulnerable to overfitting issues, whereas gen-
erative models encounter data-filling challenges and ex-
hibit inferior data scaling capabilities when compared to
contrastive models. While contrastive models often fo-
cus on global vi...

--- Hasil Pencarian 3 ---
Sumber: Self-supervic